In [1]:
import pandas as pd
import csv
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import gensim
from gensim import corpora, models
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from pprint import pprint

stemmer = SnowballStemmer("english") # the stopwords 
def remain_english(line):
    rule =re.compile("[^a-z^A-Z]")
    line = rule.sub('',line)
    return line
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) # remain problems

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(str(text)) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
            result.append(remain_english(lemmatize_stemming((token))).lower()) # take out not english and return into verb, and lower it 
    return result

C:\Users\zmz22\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\zmz22\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
# read file into pandas
import pandas as pd
fileGP = r'C:\Users\zmz22\Downloads\ToolsReview.csv'
fileApple = r'C:\Users\zmz22\Downloads\apple-utilities-reviews.csv'

df1 = pd.read_csv(fileGP)
df2 = pd.read_csv(fileApple)

C:\Users\zmz22\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (14,15,16,19,22,24,26,27,28,31,34,37,38,39,40,43,46,48,50,51,52,55,58,60) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(df1.count()['text'])
print(df2.count()['text'])

32609
306673


In [4]:
gpNum = df1.count()['text']
ApNum = df2.count()['text'] # text(reviews) number from apple
sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
# text_num = 5500
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in range(gpNum):
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in range(ApNum):
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      
                        
print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)

number of reviews text: 339282
number of neg: 83490
number of inconsistant neg reviews text(actually a pos reviews): 607
number of pos: 231696
number of inconsistant pos reviews text(actually an neg reviews): 682


In [7]:
# negtive LDA  reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 83490
Fitting Time: 1556.308346748352
Topic: 0 
Words: 0.239*"" + 0.073*"alarm" + 0.066*"video" + 0.052*"record" + 0.050*"play" + 0.040*"sound" + 0.027*"music" + 0.019*"clock" + 0.017*"wake" + 0.014*"song"
Topic: 1 
Words: 0.029*"call" + 0.028*"phone" + 0.024*"servic" + 0.022*"number" + 0.021*"month" + 0.019*"time" + 0.017*"block" + 0.017*"messag" + 0.017*"work" + 0.014*"issu"
Topic: 2 
Words: 0.079*"connect" + 0.046*"work" + 0.037*"devic" + 0.029*"camera" + 0.022*"wifi" + 0.015*"remot" + 0.014*"network" + 0.013*"view" + 0.010*"router" + 0.010*"tri"
Topic: 3 
Words: 0.020*"like" + 0.016*"page" + 0.015*"screen" + 0.014*"button" + 0.014*"keyboard" + 0.013*"search" + 0.012*"time" + 0.011*"look" + 0.011*"browser" + 0.011*"option"
Topic: 4 
Words: 0.037*"watch" + 0.034*"phone" + 0.027*"work" + 0.025*"turn" + 0.014*"like" + 0.013*"time" + 0.013*"lock" + 0.012*"light" + 0.011*"iphon" + 0.010*"notif"
Topic: 5 
Words: 0.039*"money" + 0.034*"free" + 0.024*"review"

In [8]:
# positive LDA reviews analyse
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=4)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 231696
Fitting Time: 3438.9082069396973
Topic: 0 
Words: 0.038*"awesom" + 0.034*"love" + 0.032*"like" + 0.029*"cool" + 0.028*"keyboard" + 0.022*"light" + 0.019*"great" + 0.018*"text" + 0.017*"font" + 0.014*"color"
Topic: 1 
Words: 0.027*"like" + 0.026*"star" + 0.021*"review" + 0.017*"work" + 0.016*"nice" + 0.014*"browser" + 0.013*"open" + 0.013*"better" + 0.012*"search" + 0.012*"version"
Topic: 2 
Words: 0.031*"phone" + 0.025*"call" + 0.024*"number" + 0.021*"servic" + 0.019*"thank" + 0.015*"time" + 0.012*"year" + 0.012*"custom" + 0.011*"know" + 0.011*"block"
Topic: 3 
Words: 0.391*"" + 0.054*"ipod" + 0.025*"minecraft" + 0.014*"easiest" + 0.010*"guitar" + 0.010*"percentag" + 0.009*"touch" + 0.007*"homework" + 0.007*"gboard" + 0.006*"loop"
Topic: 4 
Words: 0.119*"love" + 0.035*"best" + 0.034*"alarm" + 0.023*"time" + 0.021*"sound" + 0.017*"clock" + 0.016*"great" + 0.016*"skin" + 0.015*"like" + 0.014*"wake"
Topic: 5 
Words: 0.065*"download" + 0.032*"video" + 0.030*"file" + 0.027